# **라이브러리 로드**

In [ ]:
from math import sqrt
import pandas as pd
import numpy as np


# 데이터 전처리 패키지
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 모델 패키지
from sklearn.ensemble import RandomForestClassifier
from deap import base, creator, tools, algorithms
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dropout

# 모델 평가 패키지
from sklearn.metrics import accuracy_score

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

# **실습1 : 차원 축소 - 변수 선택법(유전 알고리즘)**

breast-cancer.csv
- 유방 종양의 30개 물리적 특성 관련 지표
- 목표 변수 : Diagnosis(악성M/양성B)

### **데이터 로드**

In [ ]:
# 범주형 변수 컬럼 확인 및 라벨인코딩 함수
def preprocess_data(data):
    # Identify categorical columns
    categorical_columns = data.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        data[col] = LabelEncoder().fit_transform(data[col])
    return data

In [ ]:
# data 로드
data = pd.read_csv("../data/breast-cancer.csv")


### **데이터 전처리**

In [ ]:
# 학습에 불필요한 독립 변수 제거 'id', 'diagnosis', 'Unnamed: 32'
X = data.drop(columns=['id', 'diagnosis', 'Unnamed: 32'], errors='ignore')
# 타겟 변수 설정 'diagnosis'
y = data['diagnosis']
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
# 데이터 정보 확인
X.info()

## 3. 유전 알고리즘 설정

In [ ]:
# 유전 알고리즘 설정
# Fitness 함수 정의 (정확도를 최대화하는 문제)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()


In [ ]:
# 유전자의 표현 (특성 선택 여부를 0 또는 1로 표시)
toolbox.register("attr_bool", np.random.randint, 2)

# 개체 생성 (특성의 수만큼 0 또는 1로 이루어진 리스트)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X.shape[1])

# 개체군 생성
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [ ]:
# 평가 함수 정의
def evaluate(individual):
    # 선택된 특성만으로 모델 학습 및 평가
    selected_features = [index for index, value in enumerate(individual) if value == 1]
    if len(selected_features) == 0:  # 특성을 하나도 선택하지 않은 경우 패널티 부여
        return 0.0,
    
    X_train_selected = X_train.iloc[:, selected_features]
    X_test_selected = X_test.iloc[:, selected_features]
    
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train_selected, y_train)
    predictions = model.predict(X_test_selected)
    accuracy = accuracy_score(y_test, predictions)
    return accuracy,

toolbox.register("evaluate", evaluate)

# 교배, 변이, 선택 연산 정의
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


In [ ]:
# 알고리즘 적용 전 초기 모델 성능 확인
initial_model = RandomForestClassifier(random_state=42)
initial_model.fit(X_train, y_train)
initial_predictions = initial_model.predict(X_test)
initial_accuracy = accuracy_score(y_test, initial_predictions)
print(f"Initial accuracy (all features): {initial_accuracy:.4f}")

In [ ]:
# 유전 알고리즘으로 변수 선택 수행
population = toolbox.population(n=50)  # 초기 개체군 크기
ngen = 20  # 세대 수
cxpb = 0.5  # 교배 확률
mutpb = 0.2  # 변이 확률

# 통계 정보 출력 설정
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

population, logbook = algorithms.eaSimple(
    population, toolbox, cxpb, mutpb, ngen, stats=stats, verbose=True
)

# 최적의 해 찾기
best_individual = tools.selBest(population, k=1)[0]
selected_features = [index for index, value in enumerate(best_individual) if value == 1]
print(f"Best individual: {best_individual}")
print(f"Selected features: {selected_features}")

In [ ]:
# 선택된 특성을 사용한 모델 정확도 계산
if len(selected_features) > 0:
    X_train_selected = X_train.iloc[:, selected_features]
    X_test_selected = X_test.iloc[:, selected_features]

    final_model = RandomForestClassifier(random_state=42)
    final_model.fit(X_train_selected, y_train)
    final_predictions = final_model.predict(X_test_selected)
    final_accuracy = accuracy_score(y_test, final_predictions)
    print(f"Final accuracy (selected features): {final_accuracy:.4f}")
else:
    final_accuracy = 0.0
    print("No features were selected.")

In [ ]:
# 개선 정확도 산출
improvement = final_accuracy - initial_accuracy
print(f"Accuracy improvement: {improvement:.4f}")

***

# **실습2: 차원 축소 - 변수 추출법(PCA)**

mnist dataset
- 수필로 작성된 숫자(0~9) 이미지 70000개로 구성된 데이터 셋
- 28x28 픽셀 크기

### **데이터 로드**

In [ ]:
# 데이터 로드 mnist.npz
data = np.load('../data/mnist.npz')
x_train, y_train = data['x_train'], data['y_train']
x_test, y_test = data['x_test'], data['y_test']

# 이미지 정규화 (scale pixel values to the range [0, 1])
x_train = x_train / 255.0  # 픽셀 값을 0~1 범위로 변환
x_test = x_test / 255.0    # 픽셀 값을 0~1 범위로 변환

# 채널 차원에 맞게 image reshape (N, H, W, C)
# CNN에 적합하도록 채널 차원 추가(CNN 입력을 위해 4차원 데이터로 변경)
x_train = x_train[..., np.newaxis]  
x_test = x_test[..., np.newaxis]    

# Datashape 확인

print('Traning data shape:', x_train.shape)
print('Testing data shape:', x_test.shape)
# Data class 확인
y_train.shape,y_test.shape

In [ ]:
# 라벨 사전 정의
label_dict = {
 0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
}

In [ ]:
# 데이터 확인을 위한 시각화
plt.figure(figsize=[5,5])  # 그래프 크기 설정

# 훈련 데이터의 첫 번째 이미지 출력
plt.subplot(121)  
curr_img = np.reshape(x_train[0], (28, 28, 1))  
plt.imshow(curr_img)  # 이미지를 표시
print(plt.title("(Label: " + str(label_dict[y_train[0]]) + ")"))  # 레이블 출력

# 테스트 데이터의 첫 번째 이미지 출력
plt.subplot(122)  
curr_img = np.reshape(x_test[0], (28, 28, 1))  
plt.imshow(curr_img)  # 이미지를 표시
print(plt.title("(Label: " + str(label_dict[y_test[0]]) + ")"))  # 레이블 출력

### **데이터 Scaling**

In [ ]:
#pca를 위한 scaling
np.min(x_train),np.max(x_train)


In [ ]:
#784개의 픽셀
x_train_flat = x_train.reshape(-1,784)
x_test_flat = x_test.reshape(-1,784)

In [ ]:
# 데이터프레임 생성
# 특징 이름 생성 (픽셀 값에 대해 열 이름 설정)
feat_cols = ['pixel'+str(i) for i in range(x_train_flat.shape[1])]  # x_train_flat의 열 수만큼 'pixel0', 'pixel1', ... 형식의 이름 생성

# MNIST 데이터를 DataFrame으로 변환
df_mnist = pd.DataFrame(x_train_flat, columns=feat_cols)  # x_train_flat 데이터를 열 이름과 함께 DataFrame으로 변환
df_mnist['label'] = y_train  # 레이블 데이터를 DataFrame의 새로운 열로 추가

# DataFrame 크기 출력
print('Size of the dataframe: {}'.format(df_mnist.shape))  # DataFrame의 크기 (행, 열) 출력

In [ ]:
df_mnist.head()

### **PCA 적용**

In [ ]:
pca_mnist = PCA(n_components=2)
principalComponents_mnist = pca_mnist.fit_transform(df_mnist.iloc[:,:-1])

In [ ]:
# 데이터 프레임 생성
principal_mnist_Df = pd.DataFrame(data = principalComponents_mnist
             , columns = ['principal component 1', 'principal component 2'])
principal_mnist_Df['y'] = y_train

principal_mnist_Df.head()

In [ ]:
# 시각화
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="principal component 1", y="principal component 2",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=principal_mnist_Df,
    legend="full",
    alpha=0.3
)


### **PCA 효과성 검증**

In [ ]:
# 데이터 주요 성분 추출
pca = PCA(0.9)

In [ ]:
pca.fit(x_train_flat)

In [ ]:
#pca수
pca.n_components_

In [ ]:
#x_train_flat: 기존 데이터
#train_img_pca: pca 적용 후 데이터
train_img_pca = pca.transform(x_train_flat)
test_img_pca = pca.transform(x_test_flat)

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## 모델 학습

In [ ]:
# 학습 파라미터 설정
batch_size = 128
num_classes = 10
epochs = 40

In [ ]:
#원본 학습
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(784,)))
model.add(Dropout(0.3))  # Dropout 추가
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))  # Dropout 추가
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train_flat, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(x_test_flat, y_test))

In [ ]:
#pca 학습
model_pca = Sequential()
model_pca.add(Dense(1024, activation='relu', input_shape=(87,)))
model_pca.add(Dropout(0.5))  # Dropout 추가
model_pca.add(Dense(1024, activation='relu'))
model_pca.add(Dropout(0.5))  # Dropout 추가
model_pca.add(Dense(512, activation='relu'))
model_pca.add(Dense(256, activation='relu'))
model_pca.add(Dense(num_classes, activation='softmax'))

model_pca.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history_pca = model_pca.fit(train_img_pca, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(test_img_pca, y_test))

## PCA 적용 유무에 따른 성능 비교

In [ ]:
# Plotting the accuracy of the two models
plt.figure(figsize=(14, 6))

# Plot training accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Original Model - Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Original Model - Validation Accuracy')
plt.plot(history_pca.history['accuracy'], label='PCA Model - Training Accuracy')
plt.plot(history_pca.history['val_accuracy'], label='PCA Model - Validation Accuracy')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()